<h2>Import Libraries</h2>

In [31]:
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split 
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.metrics import roc_curve, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy.random import randint
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
import statsmodels.api as sm

In [2]:
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [3]:
data_train = pd.read_csv('Football_data_training_new.csv', header=0)
data_test = pd.read_csv('Football_data_test_new.csv', header=0)

In [4]:
data_train = data_train.dropna()
data_test = data_test.dropna()
print("(Rows, Columns):{0}".format(data_train.shape))
print("(Rows, Columns):{0}".format(data_test.shape))

(Rows, Columns):(9036, 67)
(Rows, Columns):(1372, 67)


In [5]:
data_train.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,HDCPOH,HDCPOA,CC,PSCH,PSCD,PSCA
0,E0,8/8/2015,Bournemouth,Aston Villa,0,1,A,0.0,0.0,D,...,1.98,1.93,1.99,1.92,0.00,1.92,A,1.82,3.88,4.70
1,E0,8/8/2015,Chelsea,Swansea,2,2,D,2.0,1.0,H,...,2.24,2.16,1.80,1.73,0.00,1.73,A,1.37,5.04,10.88
2,E0,8/8/2015,Everton,Watford,2,2,D,0.0,1.0,A,...,2.28,2.18,1.76,1.71,0.00,1.71,A,1.75,3.76,5.44
3,E0,8/8/2015,Leicester,Sunderland,4,2,H,3.0,0.0,H,...,2.00,1.95,1.96,1.90,1.95,0.00,H,1.79,3.74,5.10
4,E0,8/8/2015,Man United,Tottenham,1,0,H,1.0,0.0,H,...,2.20,2.09,1.82,1.78,1.00,1.00,H,1.64,4.07,6.04


<h3>Rough Model Finding</h3>

In [44]:
for data in [data_train, data_test]:
    data['result'] = np.where(data.HDCPOH > data.HDCPOA, "Win", "Lost")
    data['result'] = np.where(data.HDCPOH == data.HDCPOA, "Draw", data['result'])

In [7]:
data_test.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAvAHH,BbMxAHA,BbAvAHA,HDCPOH,HDCPOA,CC,PSCH,PSCD,PSCA,result
0,D1,18/8/2017,Bayern Munich,Leverkusen,3,1,H,2,0,H,...,2.18,1.77,1.72,1.00,1.00,D,1.28,6.70,11.30,Draw
1,D1,19/8/2017,Hamburg,Augsburg,1,0,H,1,0,H,...,1.84,2.07,2.03,1.84,0.00,H,2.32,3.32,3.47,Win
2,D1,19/8/2017,Hertha,Stuttgart,2,0,H,0,0,D,...,2.05,1.87,1.83,2.05,0.00,H,2.02,3.60,4.07,Win
3,D1,19/8/2017,Hoffenheim,Werder Bremen,1,0,H,0,0,D,...,1.98,1.91,1.88,1.49,0.50,H,1.85,4.19,4.12,Win
4,D1,19/8/2017,Mainz,Hannover,0,1,A,0,0,D,...,1.97,1.93,1.89,0.00,1.89,A,1.85,3.75,4.65,Lost


In [116]:
attributes = list(data_train)
remove_attributes = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 
                     'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'HDCPOH', 'HDCPOA', 'result', 'CC']
cols = [x for x in attributes if x not in remove_attributes]
cols = ['BbMxAHH','BbAvAHH', 'BbAvAHA', 'PSCH', 'PSCA', 'BbMxA']
models=[MLPClassifier, LinearSVC, KNeighborsClassifier, RandomForestClassifier, ExtraTreesClassifier, LogisticRegression, QDA]

In [117]:
for model in models:
    kfold = model_selection.KFold(n_splits=5, random_state=0)
    results = model_selection.cross_val_score(model(), data_train[cols], data_train.result, cv=kfold, scoring='accuracy')
    print(model.__name__, ': ', results.mean())

MLPClassifier :  0.5511286614003555
LinearSVC :  0.5395065159580981
KNeighborsClassifier :  0.5407250441988138
RandomForestClassifier :  0.5488054688992169
ExtraTreesClassifier :  0.5405042337811168
LogisticRegression :  0.5517918884769651
QuadraticDiscriminantAnalysis :  0.5418319122782101


<h2>Multi-layer Perceptron classifier</h2>

In [123]:
MLP = MLPClassifier(hidden_layer_sizes=(20,5), alpha=0, learning_rate='adaptive', 
                    learning_rate_init=1e-4, max_iter=3000, beta_1=0.9, beta_2=0.999, random_state=0)
kfold = model_selection.KFold(n_splits=5, random_state=0)
results = model_selection.cross_val_score(MLP, data_train[cols], data_train.result, cv=kfold, scoring='accuracy')
print('Cross Validation Accuracy: ', results.mean())

Cross Validation Accuracy:  0.5502435832137558


<h2>Linear Support Vector Classification</h2>

In [124]:
Linear = LinearSVC(penalty='l1', dual=False)
kfold = model_selection.KFold(n_splits=5, random_state=0)
results = model_selection.cross_val_score(Linear, data_train[cols], data_train.result, cv=kfold, scoring='accuracy')
print('Cross Validation Accuracy: ', results.mean())

Cross Validation Accuracy:  0.5523452919080664


<h2>K-nearest neighbors</h2>

In [127]:
KNN = KNeighborsClassifier(n_neighbors=100)
kfold = model_selection.KFold(n_splits=5, random_state=0)
results = model_selection.cross_val_score(KNN, data_train[cols], data_train.result, cv=kfold, scoring='accuracy')
print('Cross Validation Accuracy: ', results.mean())

Cross Validation Accuracy:  0.5669558158782708


<h2>Random Forest</h2>

In [133]:
RF = RandomForestClassifier(n_estimators=80, min_samples_split=4, min_samples_leaf=3, random_state=0)
kfold = model_selection.KFold(n_splits=5, random_state=0)
results = model_selection.cross_val_score(RF, data_train[cols], data_train.result, cv=kfold, scoring='accuracy')
print('Cross Validation Accuracy: ', results.mean())

Cross Validation Accuracy:  0.5545593953700212


<h2>Extra Trees</h2>

In [134]:
ET = ExtraTreesClassifier(n_estimators=50, min_samples_split=20, min_samples_leaf=4, random_state=0)
kfold = model_selection.KFold(n_splits=5, random_state=0)
results = model_selection.cross_val_score(ET, data_train[cols], data_train.result, cv=kfold, scoring='accuracy')
print('Cross Validation Accuracy: ', results.mean())

Cross Validation Accuracy:  0.5569950438559975


<h2>Logistic Regression</h2>

In [135]:
LR = LogisticRegression(penalty='l1')
kfold = model_selection.KFold(n_splits=5, random_state=0)
results = model_selection.cross_val_score(LR, data_train[cols], data_train.result, cv=kfold, scoring='accuracy')
print('Cross Validation Accuracy: ', results.mean())

Cross Validation Accuracy:  0.552677395183921


<h2>Quadratic Discriminant Analysis</h2>

In [136]:
QDA_model = QDA()
kfold = model_selection.KFold(n_splits=5, random_state=0)
results = model_selection.cross_val_score(QDA_model, data_train[cols], data_train.result, cv=kfold, scoring='accuracy')
print('Cross Validation Accuracy: ', results.mean())

Cross Validation Accuracy:  0.5418319122782101


<h2>Binary Classification</h2>

<p>The result is divided into two group - 'win' and 'lost'.</p>
<ul>
    <li>'win' - The home team win after including handicup.</li>
    <li>'lost' - The home team lost/draw after including the handicup.</li>
</ul>

In [69]:
for data in [data_train, data_test]:
    data['result'] = np.where(data.HDCPOH > data.HDCPOA, 1, 0)
data_noDraw = data_train[data_train.HDCPOH!=data_train.HDCPOA]

In [82]:
X =data_noDraw[cols]
y=data_noDraw.result

In [71]:
logit_model = sm.Logit(np.asarray(y.astype(float)), np.asarray(X.astype(float)))
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.684254
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 8181
Model:                          Logit   Df Residuals:                     8139
Method:                           MLE   Df Model:                           41
Date:                Thu, 15 Mar 2018   Pseudo R-squ.:                 0.01280
Time:                        20:31:37   Log-Likelihood:                -5597.9
converged:                       True   LL-Null:                       -5670.5
                                        LLR p-value:                 1.463e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1320      0.131      1.006      0.314      -0.125       0.389
x2            -0.1526      0.

In [108]:
print(cols[35], cols[36], cols[38], cols[39], cols[41], cols[26])

IndexError: list index out of range

In [110]:
cols = ['BbMxAHH','BbAvAHH', 'BbAvAHA', 'PSCH', 'PSCA', 'BbMxA']
X_train, X_test, y_train, y_test = train_test_split(data_noDraw[cols], data_noDraw.result, test_size=0.2, random_state=0)

In [111]:
def classification_coef(index):
    classification = LogisticRegression()
    classification.fit(X_train, y_train)
    return classification.intercept_, classification.coef_

In [112]:
coef = []
intercept = []
for i in range(1000):
    new_intercept, new_coef = classification_coef(index=randint(len(X), size=len(X)))
    coef.append(new_coef.tolist()[0])
    intercept.append(new_intercept.tolist()[0])
    if (i%50==0):
        print(i/10, '%')

0.0 %
5.0 %
10.0 %
15.0 %
20.0 %
25.0 %
30.0 %
35.0 %
40.0 %
45.0 %
50.0 %
55.0 %
60.0 %
65.0 %
70.0 %
75.0 %
80.0 %
85.0 %
90.0 %
95.0 %


In [137]:
print('Intercept [Mean]: ', np.mean(intercept))
print('Intercept [SD]: ', np.std(intercept))
print('Coefficient [Mean]: ', np.mean(coef, axis=0))
print('Coefficient [SD]: ', np.std(coef, axis=0))

Intercept [Mean]:  -0.059087048902554555
Intercept [SD]:  2.0816681711721685e-17
Coefficient [Mean]:  [ 0.34279878 -0.78415802  0.42039228 -0.00652029  0.04913552 -0.02999692]
Coefficient [SD]:  [3.77475828e-15 1.33226763e-14 2.10942375e-15 9.97465999e-17
 1.00613962e-15 5.13478149e-16]


In [114]:
classification = LogisticRegression()
classification.fit(X_train, y_train)
classification.score(X_test, y_test)

0.5271838729383018